In [45]:
from Bio.Seq import Seq
from Bio import pairwise2
from Bio.pairwise2 import format_alignment
import pandas as pd

In [46]:
def find_mutations(wt, mut, barcode_idx_end, verbose=False):
    """Outputs list of mutations from wild type to mutant"""
    alignments = pairwise2.align.globalms(wt, mut, 2, -1, -5, -0.1)
    aln = format_alignment(*alignments[0])
    if verbose:
        print(aln)
    start1 = [aln.find(" "), aln.find("|"), aln.find(".")]
    start = min(start1)
    mutations = []
    ins_count = 0
    for i in range(0, start):
        if aln[i] == "-":
            ins_count += 1
        if aln[start + i] == ".":
            g = aln[i]
            m = aln[start * 2 + i]
            pos = i - ins_count + 1
            if m != "-":
                mutations.append(g + str(pos) + m)
    return mutations


def find_mutations_noalign(wt, mut, barcode_idx_end, verbose=False):
    """Output list of mutations from wild type to mutant by using barcode index."""
    mutations = []
    for i in range(barcode_idx_end + 1, len(wt)):
        if wt[i] != mut[i]:
            mutations.append(wt[i] + str(i + 1) + mut[i])
    return mutations


def find_closest_gene(fmfunc, wts, mut, verbose=False):
    mut_sets = []
    for wt in wts:
        mut_sets.append(fmfunc(wt, mut, 3, verbose))
    mut_len = [len(mut_set) for mut_set in mut_sets]
    minimum = mut_len.index(min(mut_len))
    return mut_sets, minimum, wts[minimum]


def min_hamming_distance(mut, wts, print_align=False):
    distances = []
    for wt in wts:
        str1 = mut
        if len(str1) < len(wt):
            str1 = str1 + "-" * (len(wt) - len(str1))
            if print_align:
                print(str1 + "\n" + wt)
        elif len(str1) > len(wt):
            wt = wt + "-" * (len(str1) - len(wt))
            if print_align:
                print(str1 + "\n" + wt)
        else:
            if print_align:
                print(str1 + "\n" + wt)
        distances.append(sum(c1 != c2 for c1, c2 in zip(str1, wt)))
    min_idx = distances.index(min(distances))

    return min_idx, wts[min_idx]


wts = ["ATGCATGC", "ATGCCCCCAT", "ATCCAACC", "ATGCATCCAAAATTTT", "ATCCAACCAAAG"]
mut = "ATCCAACCAAAA"

result = min_hamming_distance(mut, wts, print_align=True)
print("Index of min distance WT: ", result[0])
print("Matched WT: ", result[1])

ATCCAACCAAAA
ATGCATGC----
ATCCAACCAAAA
ATGCCCCCAT--
ATCCAACCAAAA
ATCCAACC----
ATCCAACCAAAA----
ATGCATCCAAAATTTT
ATCCAACCAAAA
ATCCAACCAAAG
Index of min distance WT:  4
Matched WT:  ATCCAACCAAAG
